# Python con BBDD

Hasta ahora hemos aprendido sobre programación usando python y los comandos más usados de SQL.

Ahora veremos como montar una base de datos en la nube y manipularla usando python a nuestro gusto.

Para esto usaremos

```
import psycopg2
```

Una libreria enfocada en la manipulación de bases de datos usando python.

Vamos ir viendo paso a paso como se utiliza.


In [ ]:
import psycopg2

#Conectarse a la Base de Datos
con = psycopg2.connect(
    database="",
    user="",
    password="",
    host="",
    port="")


print("Base de datos Conectada")

#Abrir la opción de ejecutar una linea SQL
cur = con.cursor()

#Escribir la linea de código SQL
cur.execute('SQL')

#Ejecutar el comando
con.commit()

#Cerrar la conexión
con.close()


Esta es la forma más simple de ejecutar postgres usando python.

Llevemoslo a la realidad.

Para esto usaremos una plataforma gratuita llamada Heroku y creemos la tabla users.

In [ ]:
import psycopg2

#Conectemonos a Heroku
con = psycopg2.connect(
    database="",
    user="",
    password="",
    host="",
    port="")


print("Base de datos Conectada")





Base de datos Conectada


In [ ]:
#Abrir la opción de ejecutar una linea SQL
cur = con.cursor()

#Escribir la linea de código SQL
cur.execute('''CREATE TABLE users  (
      user_id SERIAL PRIMARY KEY ,
      first_name VARCHAR(255) NOT NULL,
      last_name VARCHAR(255) NOT NULL,
      email VARCHAR(255) NOT NULL UNIQUE,
      phone VARCHAR(255) NOT NULL UNIQUE
    );

    CREATE TABLE posts (
        post_id SERIAL PRIMARY KEY ,
        title varchar(255) NOT NULL,
        content TEXT NOT NULL,
        user_id INTEGER,
        FOREIGN KEY (user_id) 
          REFERENCES users (user_id) 
            ON DELETE CASCADE 
            ON UPDATE NO ACTION
    );

    CREATE TABLE comments (
        comment_id SERIAL PRIMARY KEY ,
        content varchar(255) NOT NULL,
        user_id INTEGER NOT NULL,
        post_id INTEGER NOT NULL,
        FOREIGN KEY (post_id) 
          REFERENCES posts (post_id) 
            ON DELETE CASCADE 
            ON UPDATE NO ACTION,
        FOREIGN KEY (user_id) 
          REFERENCES users (user_id) 
            ON DELETE CASCADE 
            ON UPDATE NO ACTION
    );
        
    ''')

#Ejecutar el comando
con.commit()

print("Ejecutado")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 4))



ProgrammingError: ignored

Insertemos Datos

In [ ]:
import psycopg2



cur = con.cursor()

postgres_insert_query = """INSERT INTO
    users (first_name,last_name,email,phone)
    VALUES (%s,%s,%s,%s)"""
record_to_insert = ['Pepito','Hurtado','a123hga@gmail.com','682669891']

cur.execute(postgres_insert_query,record_to_insert)

#Ejecutar el comando
con.commit()

IntegrityError: ignored

Cambiemos Datos

In [ ]:
import psycopg2



cur = con.cursor()

postgres_update_query = """UPDATE users set first_name = %s where User_id = %s"""
record_to_update = ['Pepito2',5]

cur.execute(postgres_update_query,record_to_update)

#Ejecutar el comando
con.commit()

Mostremos Datos

In [ ]:
import psycopg2
import pandas as pd



cur = con.cursor()

cur.execute(
    """SELECT * from users"""
)
con.commit()
rows = cur.fetchall()

# for row in rows:
#     print("Nombre =", row[1])
#     print("Apellido =", row[2])
#     print("Email =", row[3])
#     print("Celular =", row[4], "\n")

com_frame = pd.DataFrame(rows)
print(com_frame)





    0        1        2                  3           4
0  11   Pepito  Hurtado  a123hga@gmail.com   682669891
1   5  Pepito2  Hurtado    a12hg@gmail.com   781669891
2   4  Pepito1  Hurtado    p12hg@gmail.com  7816693891
3   6  Pepito1  Hurtado   a12hga@gmail.com   681669891
4   3  Pepito1  Hurtado     p1hg@gmail.com   786693891


Borremos Datos

In [ ]:
import psycopg2

#Conectemonos a Heroku


cur = con.cursor()

postgres_delete_query = """Delete from
    users where
    user_id = %s"""
record_to_delete = [1]

cur.execute(postgres_delete_query,record_to_delete)

#Ejecutar el comando
con.commit()

Esto es lo más básico que se puede hacer usando esta libreria, pero nosotros y nosotras podemos hacerlo mejor.

In [ ]:
import psycopg2


class db:
  def __init__(self,database,user,password,host,port):
    self.database = database
    self.user = user
    self.password = password
    self.host = host
    self.port = port
    self.connection = None


  def connect(self):
    self.connection = psycopg2.connect(
    database= self.database,
    user = self.user,
    password= self.password,
    host= self.host,
    port= self.port
    )

    return self

  def show_tables(self):

    cur = self.connection.cursor()
    cur.execute('''
      SELECT table_name
      FROM information_schema.tables
      WHERE table_schema = 'public'
      ORDER BY table_name;
    ''')
    rows = cur.fetchall()

    for row in rows:
        print(row[0])
      
    self.connection.commit()

    return self

  def select(self,table):
    cur = self.connection.cursor()
    
    postgres_select_query = "Select * from "+table
 

    cur.execute(postgres_select_query)

    rows = cur.fetchall()

    for row in rows:
      for value in row:
        print(value, end=" ,")
      print("---\n")
        
      
    self.connection.commit()


database="d6pco1m2d1b36c"
user="tkfxjiwjmvumfh"
password="9bfd73df88c2a742cafe819b7648e72b41d029dac1e453fef48b01079ceaecbb"
host="ec2-34-192-173-173.compute-1.amazonaws.com"
port="5432"

db = db(database,user,password,host,port)
db.connect().show_tables()
db.select("users")

comments
posts
users
4 ,Pepito1 ,Hurtado ,p12hg@gmail.com ,7816693891 ,---

6 ,Pepito1 ,Hurtado ,a12hga@gmail.com ,681669891 ,---

3 ,Pepito1 ,Hurtado ,p1hg@gmail.com ,786693891 ,---

5 ,Pepito2 ,Hurtado ,a12hg@gmail.com ,781669891 ,---

